# <div align="center" style="color: #ff5733;">Data Requested By Annastasia for Monthly Reporting</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 5

In [3]:
# How many Applications applied to each month

sq = """
# loan applied
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    case when reloan_flag = 1  and loantype not like 'FLEXUP' then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
         
         else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1 desc,2;
"""

StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in StartedApps are:\t {StartedApps.shape}")

Job ID 66ef64b4-bc43-4ec6-bd28-6a46d8aa5f0d successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in StartedApps are:	 (182, 3)


In [4]:
# How many submitted applications to each month
sq = """
# loan Submitted
SELECT 
    DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime else termsAndConditionsSubmitDateTime end),MONTH) as mm,  
    case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
group by 1,2
order by 1 desc,2;
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of SubmittedApps are:\t {SubmittedApps.shape}")
SubmittedApps.sample(10)

Job ID a12c5b5d-abb3-4cca-8270-a56882eda430 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns of SubmittedApps are:	 (183, 3)


,mm,LoanProduct,SubmittedApps
162,2021-11-01,Flex,2396
138,2022-06-01,SIL-Instore,124
32,2024-01-01,Flex-up,347
90,2023-03-01,Big Loan,22
7,2024-05-01,Flex,18
148,2022-03-01,Flex,2894
65,2023-08-01,SIL-Instore,3918
83,2023-05-01,SIL-Instore,1405
152,2022-02-01,Flex,2395
113,2022-11-01,Big Loan,4


In [5]:
# Approved Loans for each month

sq = """
# Approved loans
SELECT 
DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime 
                  when reloan_flag = 1 and loantype not like 'FLEXUP' then startApplyDateTime else decision_date end),MONTH) as mm,
        case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where 
(case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then flagDisbursement 
      when reloan_flag = 1 and loantype not like 'FLEXUP' then flagDisbursement  else flagApproval end) = 1
group by 1,2
order by 1 desc,2
;
"""

ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps are:\t {ApprovedApps.shape}")
ApprovedApps.head(10)

Job ID 5dc68f9c-2c06-4ae7-adbc-ebb784ea6a5d successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps are:	 (150, 3)


,mm,LoanProduct,ApprovedApps
0,2024-06-01,Flex-up,15
1,2024-06-01,Quick,83
2,2024-06-01,Reloan,9
3,2024-06-01,SIL-Instore,1159
4,2024-05-01,ACL TSA,2
5,2024-05-01,Flex-up,438
6,2024-05-01,Quick,1142
7,2024-05-01,Reloan,310
8,2024-05-01,SIL-Instore,10186
9,2024-04-01,ACL TSA,1


In [6]:
# Booked Loans for each month

sq = """
#Booked loans
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
            case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where flagDisbursement = 1

group by 1,2
order by 1 desc,2
;
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t {BookedApps.shape}")
BookedApps.head(10)

Job ID cb764b63-c0ce-4e57-ae01-9cc983327b77 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	 (148, 3)


,mm,LoanProduct,BookedApps
0,2024-06-01,Flex-up,32
1,2024-06-01,Quick,80
2,2024-06-01,Reloan,9
3,2024-06-01,SIL-Instore,1147
4,2024-05-01,ACL TSA,2
5,2024-05-01,Flex-up,525
6,2024-05-01,Quick,1111
7,2024-05-01,Reloan,310
8,2024-05-01,SIL-Instore,10177
9,2024-04-01,ACL TSA,1


In [7]:
# Get the total booked amount for a month

sq = """
# Booked Amount
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
     where flagDisbursement = 1
group by 1,2
order by 1 desc,2
;
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t {BookedAmt.shape}")
BookedAmt.head(10)

Job ID 3187d036-3112-41a6-957f-37848465b92b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	 (148, 3)


,mm,LoanProduct,BookedAmt
0,2024-06-01,Flex-up,1.318188e+06
1,2024-06-01,Quick,1.126500e+06
2,2024-06-01,Reloan,1.608409e+05
3,2024-06-01,SIL-Instore,1.129666e+07
4,2024-05-01,ACL TSA,4.000000e+04
5,2024-05-01,Flex-up,2.160140e+07
6,2024-05-01,Quick,1.449350e+07
7,2024-05-01,Reloan,9.638664e+06
8,2024-05-01,SIL-Instore,1.026048e+08
9,2024-04-01,ACL TSA,2.000000e+04


In [8]:
BookedAmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   mm           148 non-null    datetime64[us]
 1   LoanProduct  148 non-null    object        
 2   BookedAmt    148 non-null    float64       
dtypes: datetime64[us](1), float64(1), object(1)
memory usage: 3.6+ KB


In [9]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [10]:
raw_data.tail(10)

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
175,2024-05-01,Flex-up,440,438,438,525,2.160140e+07
176,2024-05-01,Quick,33872,24636,1142,1111,1.449350e+07
177,2024-05-01,Reloan,339,324,310,310,9.638664e+06
178,2024-05-01,SIL-Instore,18485,18222,10186,10177,1.026048e+08
179,2024-06-01,ACL TSA,2,2,<NA>,<NA>,NaN
180,2024-06-01,Flex,1,1,<NA>,<NA>,NaN
181,2024-06-01,Flex-up,15,15,15,32,1.318188e+06
182,2024-06-01,Quick,2792,2094,83,80,1.126500e+06
183,2024-06-01,Reloan,11,11,9,9,1.608409e+05
184,2024-06-01,SIL-Instore,1748,1733,1159,1147,1.129666e+07


In [11]:
print(f'{data_year}-{data_month:02d}-01')

2024-05-01


In [12]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

In [13]:
raw_data_current_month

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
53,2022-08-01,Flex,5558,4732,594,599,1.198500e+07
54,2022-08-01,Quick,43764,30660,1910,1926,3.300100e+07
55,2022-08-01,SIL-Instore,967,956,456,457,1.137941e+07
56,2022-08-01,SIL-Online(discontinued),1192,652,41,41,1.197499e+06
57,2022-09-01,Big Loan,13,12,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...
174,2024-05-01,Flex,43,18,<NA>,<NA>,NaN
175,2024-05-01,Flex-up,440,438,438,525,2.160140e+07
176,2024-05-01,Quick,33872,24636,1142,1111,1.449350e+07
177,2024-05-01,Reloan,339,324,310,310,9.638664e+06


### Loan Account Tagging

In [14]:
sq="""
# Loan Tagging
SELECT
    loanAccountNumber,
    Case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
;
"""
loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type are: {loan_type.shape}")
loan_type.tail(10)

Job ID 218ebe01-b6cd-4eba-b85c-5a936697d2ad successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type are: (139657, 4)


,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
139647,60825451720015,SIL-Instore,2024-06-01 19:52:10,None
139648,60825452290015,SIL-Instore,2024-06-01 20:36:41,None
139649,60825457150014,SIL-Instore,2024-06-02 09:22:17,None
139650,60825464600014,SIL-Instore,2024-06-02 13:55:57,None
139651,60825469940019,SIL-Instore,2024-06-02 16:20:40,None
139652,60825470080012,SIL-Instore,2024-06-02 16:26:20,None
139653,60825470310011,SIL-Instore,2024-06-02 16:32:24,None
139654,60825471300013,SIL-Instore,2024-06-02 16:56:04,None
139655,60825472580018,SIL-Instore,2024-06-02 17:26:04,None
139656,60825473300019,SIL-Instore,2024-06-02 17:46:56,None


In [15]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [16]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [17]:
sq = """
# Vas 
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
    Case when reloan_flag = 1 and b.loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when b.loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1 desc,2 desc,3
;
"""

vas_df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in vas_df are:\t {vas_df.shape}")
vas_df.head(10)

Job ID ad387469-c96a-4224-a481-714925f570f9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df are:	 (148, 5)


,year,month,new_loan_type,TotalSold,SoldWithVAS
0,2024,6,Flex-up,32,0
1,2024,6,Quick,80,55
2,2024,6,Reloan,9,0
3,2024,6,SIL-Instore,1147,832
4,2024,5,ACL TSA,2,2
5,2024,5,Flex-up,525,0
6,2024,5,Quick,1111,628
7,2024,5,Reloan,310,0
8,2024,5,SIL-Instore,10177,7285
9,2024,4,ACL TSA,1,0


In [18]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [19]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v2.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)